In [4]:
import cv2
import mediapipe as mp
import csv 
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [1]:
import os
videoFilesPath = os.getcwd()+"/videos/"
videoFiles = os.listdir(videoFilesPath)

csvFilesPath = os.getcwd()+"/csv_files/"

infoDict = {}
num_coords = 33
landmarks = []
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

for file in videoFiles:
    pre, ext = os.path.splitext(file)
    csv_name = pre+".csv"
    csv_file = csvFilesPath+csv_name
    
    if not os.path.exists(csv_file):
      print(csv_file)

      if pre[-11:] == "geschnitten":

          with open(csv_file, mode='w', newline='') as f:
              csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
              csv_writer.writerow(landmarks)

          #per vid
          video_name = os.getcwd()+"/videos/"+pre+".mp4"

          cap = cv2.VideoCapture(video_name)

          with open(csv_file, mode='w+', newline='') as f:
              csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
              csv_writer.writerow(landmarks)
              
          frame_count = 0
          safe_img_widest = None
          with mp_pose.Pose(
              min_detection_confidence=0.5,
              min_tracking_confidence=0.5) as pose:
            while cap.isOpened():
              success, image = cap.read()
              if not success:
                print("Ignoring empty camera frame.")
                print(frame_count)
                #cv2.imshow("Widest IMG", safe_img_widest)
                # If loading a video, use 'break' instead of 'continue'.
                infoDict[pre] = frame_count
                break
              else:
                frame_count +=1
              # if frame_count == 500:
              #   break
              # To improve performance, optionally mark the image as not writeable to
              # pass by reference.
              image.flags.writeable = False
              image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
              results = pose.process(image)

              # Draw the pose annotation on the image.
              image.flags.writeable = True
              font = cv2.FONT_HERSHEY_SIMPLEX
              image_hight, image_width, _ = image.shape
              """
              x_knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x #* image_width
              y_knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y #* image_hight
              x_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x #* image_width
              y_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y #* image_hight
              difference = x_knee-x_ankle"""

              image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
              mp_drawing.draw_landmarks(
                  image,
                  results.pose_landmarks,
                  mp_pose.POSE_CONNECTIONS,
                  landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

              # Export coordinates
              try:
                  # Extract Pose landmarks
                  pose_results = results.pose_landmarks.landmark
                  pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_results]).flatten())
                  
                  # Concate rows
                  row = pose_row
                  
                  # Export to CSV
                  with open(csv_file, mode='a', newline='') as f:
                      csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                      csv_writer.writerow(row) 
                  
              except Exception as e:
                  print(e)
                  pass

              # Flip the image horizontally for a selfie-view display.
              """image = cv2.putText(image, f'Audio: {audio_frame}', (50,50), font, 
                            1, (0,0,0), 4, cv2.LINE_AA)
              image = cv2.putText(image, f'Linker Knöchel X,Y: {x_ankle}, {y_ankle}', (50,100), font, 
                            1, (0,0,0), 4, cv2.LINE_AA)

              if difference > widest:
                widest = difference
                safe_img_widest = image.copy()"""


              #Video mit landmarks laufen lassen
              #cv2.imshow("Bildauschnitt", image)

              if cv2.waitKey(5) & 0xFF == 27:
                infoDict[pre] = frame_count
                break
              
          cap.release()
          cv2.destroyAllWindows()

c:\Users\robin\WI07_Repos\running_pose_analytics/csv_files/Gehen-1.csv
c:\Users\robin\WI07_Repos\running_pose_analytics/csv_files/Gehen-10.csv
c:\Users\robin\WI07_Repos\running_pose_analytics/csv_files/Gehen-10_hinten.csv
c:\Users\robin\WI07_Repos\running_pose_analytics/csv_files/Gehen-1_hinten.csv
c:\Users\robin\WI07_Repos\running_pose_analytics/csv_files/Gehen-2.csv
c:\Users\robin\WI07_Repos\running_pose_analytics/csv_files/Gehen-2_geschnitten.csv


NameError: name 'csv' is not defined

In [7]:
# num_coords = len(results.pose_landmarks.landmark)
# num_coords

In [8]:
# landmarks = ['class']
# for val in range(1, num_coords+1):
#     landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
# landmarks

In [9]:
# import pandas as pd
# import plotly.express as px

# df_side = pd.read_csv("Gehen-6_5_Seite.csv")
# df_back = pd.read_csv("Gehen-6_5_Hinten.csv")
# fig_side = px.line(df_side, y="x25", title='Position LEFT_HEEL')
# fig_back = px.line(df_back, y="z25")
# fig_back.update_traces(line= dict(
#     color = "green"
# ))

# fig_side.add_trace(fig_back.data[0], )
# fig_side.show()
# #df